In [99]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb

In [100]:
sys.path.append('..')
from helper import *

setwd()
Objects = Objects()

In [101]:
Objects

[{'id': 'eurostat',
  'format': 'xlsx',
  'landing_temp_name': 'prc_hicp_midx__custom_.*_spreadsheet.xlsx',
  'landing_temp_tblID': 'prc_hicp_midx__custom_(.*)_spreadsheet.xlsx'},
 {'id': 'ine',
  'format': 'csv',
  'delim': ';',
  'landing_temp_name': '50918.csv'}]

In [84]:
# connection to duckdb
con = duckdb.connect(database="data/formatted/db_formatted.db", read_only=False)

In [91]:
con.close()

## select files to process

In [97]:
# files already processed:
processedFiles = []
with open('logs/processed_data_persistant.txt', "r") as f0:
    for line in f0:
        processedFiles.append(line.rstrip())
processedFiles

[]

In [86]:
# files in the temporal zone:
for datasource in Objects:
    id = datasource["id"]
    print(id)
    print(os.listdir(f'data/landing_pers/{id}'))

eurostat
['eurostat_3484209_20221002_015650.xlsx', 'eurostat_3484218_20221002_015650.xlsx', 'eurostat_3484234_20221002_015650.xlsx']
ine
['ine_20221002_015650.csv']


In [96]:
# inputs:
# - Datasource: element of the Objects.json dictionary
# - Database: opened duckdb connection
# it stores the tables in the landing zone in Database
def ProcessCSV(Datasource, ProcessedFiles = [], LogWriter = ""):

    id = datasource["id"]
    # csv delimiter
    delim = datasource["delim"] if "delim" in datasource else ","
    # open database connection
    con = duckdb.connect(database=f"data/formatted/db_{id}.db", read_only=False)


    # files to process:
    files = [file for file in os.listdir(f'data/landing_pers/{id}') if not file in processedFiles]
    
    
    # TODO select only files that have not been yet processed
    for file in files:
        tablename = file.split(".")[0]
        con.execute(f"""
            CREATE TABLE test_{tablename}
            AS SELECT * FROM read_csv_auto(
                './data/landing_pers/{id}/{file}',
                HEADER = TRUE,
                DELIM = '{delim}'
            ) 
            """)
    
    # close database
    con.close()

In [90]:
for datasource in Objects:

    id = datasource["id"]
    print(id)

    format = datasource["format"]
    
    if format == "csv":
        ProcessCSV(datasource, con)
    elif format == "xlsx":
        ProcessXLSX(datasource)
    else:
        print(f"Format {format} is not supported\nSupported formats: csv, xlsx")

eurostat
ine


In [103]:
import pandas as pd
pd.read_excel('data/landing_pers/eurostat/eurostat_3484209_20221002_015650.xlsx')

d:\01-Cole\00Master\Q3-ADSDB\ADSDB_project1\venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,HICP - monthly data (index) [PRC_HICP_MIDX__cu...,NaN,NaN,NaN
5,Open product page,Open in Data Browser,NaN,NaN
6,Description:,-,NaN,NaN
7,NaN,NaN,NaN,NaN
8,Last update of data:,NaN,NaN,30/09/2022 11:00
9,Last change of data structure:,NaN,NaN,30/09/2022 11:00


In [89]:
import pandas as pd
def ProcessXLSX(Datasource):
    id = Datasource["id"]
    
    # TODO select only files that have not been yet processed
    for table in os.listdir(f'data/landing_pers/{id}'):
        pass
        #df = pd.read_excel(f'data/landing_pers/{id}/{table}')
        #display(df.head())

ProcessXLSX(Objects[0])

## useful queries to debug

In [62]:
con.execute("SHOW TABLES")
con.fetchall()

[('test',), ('test_ine_20221001_231336',)]

In [64]:
con.execute("DESCRIBE test_ine_20221001_231336")
con.fetchall()

[('Provincias', 'VARCHAR', 'YES', None, None, None),
 ('Grupos ECOICOP', 'VARCHAR', 'YES', None, None, None),
 ('Tipo de dato', 'VARCHAR', 'YES', None, None, None),
 ('Periodo', 'VARCHAR', 'YES', None, None, None),
 ('Total', 'VARCHAR', 'YES', None, None, None)]

In [68]:
con.execute("DROP TABLE test_ine_20221001_231336")

In [71]:
# con.execute()
con.description
con.fetchall()

[(686244,)]

In [72]:
con.execute("SELECT * FROM test LIMIT 10")
con.fetchall()

In [80]:
con.close()